In [1]:
from SRS import SRS
import llm as L
from llama_cpp import Llama
import re

In [2]:
srs = SRS()
srs.add_card("fox")
print(srs.get_card("fox"))
print(srs.get_reps("fox"))
srs.review_card("fox", "good")
print(srs.get_card("fox"))
print(srs.get_reps("fox"))
srs.output()

0
1


In [3]:
def logits_processor(prev_tok_ids, next_tok_logits):
    
    # print(prev_tok_ids, next_tok_logits)

    next_tok_logits[995] -= 1000
 
    return next_tok_logits

In [4]:
chat_llm = L.LLM('You are language chatbot. You speak using low-level, easy to comprehend words. You end every sentence with a short quiz to see if they understand')
grader_llm = L.LLM('You grade the responses of a user input and flag the words that are used incorrectly')


Initializing Global LLM Instance


In [5]:
user_input = input()
resp = chat_llm(user_input, max_tokens=200, logits_processor=[logits_processor])
resp

"  Sure, I'd be happy to help! Brazil is a country located in South America. It is the largest country in both South America and the Latin American region. Brazil is known for its diverse culture, rich history, and beautiful landscapes.\nCan you tell me the capital of Brazil? 🤔"

In [13]:
mistakes_dictionary = {'word':'reason'}
mistakes = grader_llm(user_input, mistakes_dictionary)
grader_llm.get_pretty_hist()

"system --- You grade the responses of a user input and flag the words that are used incorrectly\n__________\n\nuser --- Hello, what can you tell me about Brazil?\n__________\n\nassistant --- {'word': 'reason'}\n__________\n\nuser --- Hello, what can you tell me about Brazil?\n__________\n\nassistant --- {'word': 'reason'}\n__________\n\nuser --- Hello, what can you tell me about Brazil?\n__________\n\nassistant --- {'word': 'reason'}\n__________\n\nuser --- Hello, what can you tell me about Brazil?\n__________\n\nuser --- Hello, what can you tell me about Brazil?\n__________\n\nuser --- Hello, what can you tell me about Brazil?\n__________\n\nassistant --- {'word': 'reason'}\n__________\n\n"

In [7]:
#[^\w\s\-]*
output_words = re.sub("[^A-Za-z0-9 -]", "", resp)

for i in output_words:
    srs.review_card(i, 'again')
# TODO: review card instead of adding
#       'good' or 'bad'?
#       how to space words? language dictionary?
# Add Lucas's model
# add bad but not very bad
# LLM to extract incorrect words (structured output)
# LLM to break words apart betterer

In [8]:
srs.output()